
## Building a Conversational Chatbot with Custom Data 

This notebook guides you through the creation of a chatbot tailored to your specific data needs. Utilizing HuggingFaceEmbeddings and FAISS, the project transforms documents into vectors for a local vector storage system. Then it integrates the "meta-llama/llama-2-7b-chat" model from your local machine. The `langchain` library plays a crucial role in this process, aiding in tasks like chunking documents, indexing data in vector db, managing conversation chains with memory buffers, and crafting prompt templates.


### Key Features:

- **PDF Content Processing**: When users upload PDF files, the notebook extracts the text, segments it into manageable chunks, and indexes these chunks in in a vector db locally using HuggingFaceEmbeddings and FAISS.
- **Data-Driven Query Handling**: Users can pose questions to the chatbot, which searches the indexed data for relevant answers.
- **Integrating Vector Database and LLMs**: We leverage `langchain`'s capabilities to link vector database indexing with llama-2 LLMs, enabling a seamless conversational experience with memory and retrieval functionalities.
- **Hallucination Check**: The notebook includes a mechanism to detect and correct any hallucinations or inaccuracies in the LLM's responses.

### Prerequisites for Running the Notebook:


1. **Library Requirements**: Confirm that you have installed all libraries specified in the `requirements.txt` file.



In [ ]:
!pip install pypdf langchain langchain-community tiktoken llama-cpp-python panel streamlit

In [4]:
pip install htmltemplate


Note: you may need to restart the kernel to use updated packages.


Below cell imports the required libraries to run this notebook.

In [1]:

import PyPDF2

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import LlamaCpp
from langchain_community.llms import Ollama


from langchain.embeddings import HuggingFaceEmbeddings # import hf embedding
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer, util
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler



/Users/aboniasojasingarayar/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Enter your pdf file name below


In [2]:
pdf_docs=["./data/CV.pdf"]

### Step 1: Prepare above documents and their metadata
The prepare_docs function below processes a list of PDF documents by extracting text from each page and organizing it into two lists: one for the text content and another for the metadata (titles). It iterates through each page of each PDF, extracts the text, and forms a title using the PDF name and page number. The function returns these two lists, making it useful for indexing and referencing the content of multiple PDFs at a page level.

In [3]:

def prepare_docs(pdf_docs):
    docs = []
    metadata = []
    content = []

    for pdf in pdf_docs:

        pdf_reader = PyPDF2.PdfReader(pdf)
        for index, text in enumerate(pdf_reader.pages):
            doc_page = {'title': pdf + " page " + str(index + 1),
                        'content': pdf_reader.pages[index].extract_text()}
            docs.append(doc_page)
    for doc in docs:
        content.append(doc["content"])
        metadata.append({
            "title": doc["title"]
        })
    print("Content and metadata are extracted from the documents")
    return content, metadata



### Step 2: Chunk the documents 
The get_text_chunks function takes text content and metadata as inputs and splits the content into smaller chunks. It uses a RecursiveCharacterTextSplitter configured with a specified chunk size (512 characters) and overlap (256 characters) for this purpose. The function processes the content, splitting it into passages while maintaining associated metadata. After splitting, it prints the total number of passages created and returns these split documents. This function is useful for breaking down large text into more manageable, indexed segments for easier processing and retrieval.

In [4]:
def get_text_chunks(content, metadata):
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,
        chunk_overlap=256,
    )
    split_docs = text_splitter.create_documents(content, metadatas=metadata)
    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs


### Step 3: Ingest into Vector Database locally

The `ingest_into_vectordb` function is designed for processing and indexing a collection of documents into a vector database using FAISS (Facebook AI Similarity Search) for efficient similarity searches. It operates as follows:

1. **Embedding Creation**: It generates embeddings for the input documents (`split_docs`) using the Hugging Face model `'sentence-transformers/all-MiniLM-L6-v2'`. This model is specifically chosen for its efficiency in creating sentence-level embeddings and is set to run on the CPU.

2. **Vector Database Indexing**: Utilizes the generated embeddings to create a FAISS vector database. FAISS is used for its ability to efficiently handle large-scale similarity searches and clustering of dense vectors.

3. **Local Storage**: After creating the vector database, the function saves it locally to the path specified by `DB_FAISS_PATH`, ensuring the data can be easily accessed for future similarity searches or retrieval tasks.

The primary purpose of this function is to transform textual data into a structured, searchable vector format, facilitating efficient and scalable retrieval tasks such as document similarity searches or clustering.

In [5]:
def ingest_into_vectordb(split_docs):
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})
    db = FAISS.from_documents(split_docs, embeddings)

    DB_FAISS_PATH = 'vectorstore/db_faiss'
    db.save_local(DB_FAISS_PATH)
    return db

### Step 4: Set up Conversation Chain using LLM
The `get_conversation_chain` function is designed to create and configure a conversational chain for a language model, specifically using the LLaMA model and a vector database for retrievals. Here's a summary of its main components and functionalities:

1. **Callback Manager Setup**:
   - Initializes a `CallbackManager` with `StreamingStdOutCallbackHandler()`, which likely handles streaming and logging outputs during the model's operation.

2. **LLaMA Model Configuration**:
   - Instantiates a `LlamaCpp` model with specified parameters such as `model_path`, `temperature`, `max_tokens`, `top_p`, and `n_ctx`. These parameters configure the behavior of the LLaMA model, including its conversational style and technical constraints.
   - Integrates the `callback_manager` with the LLaMA model, allowing for additional processing or logging during the model's operation.

3. **Retriever Initialization**:
   - Transforms the input `vectordb` into a retriever, enabling it to fetch relevant information from the vector database during conversations.

4. **Conversation Chain Creation**:
   - Sets up a `ConversationBufferMemory`, which manages the conversation history and assists in generating context-aware responses.
   - Constructs a `ConversationalRetrievalChain` using the LLaMA model (`llama_llm`), the retriever, and the conversation memory. This chain is responsible for handling the flow of the conversation, including retrieving relevant information and generating responses.

5. **Return Value**:
   - Outputs a message indicating the successful creation of the conversational chain.
   - Returns the `conversation_chain` object, which can be used to handle conversational interactions using the LLaMA model and the vector database.

This function sets up a sophisticated conversational AI system combining the LLaMA model for language generation and a vector database for information retrieval, enhanced with a callback manager for additional processing and a conversation memory buffer for context management.

In [6]:
template = """[INST]
As an AI, provide accurate and relevant information based on the provided document. Your responses should adhere to the following guidelines:
- Answer the question based on the provided documents.
- Be direct and factual, limited to 50 words and 2-3 sentences. Begin your response without using introductory phrases like yes, no etc.
- Maintain an ethical and unbiased tone, avoiding harmful or offensive content.
- If the document does not contain relevant information, state "I cannot provide an answer based on the provided document."
- Avoid using confirmatory phrases like "Yes, you are correct" or any similar validation in your responses.
- Do not fabricate information or include questions in your responses.
- do not prompt to select answers. do not ask me questions
{question}
[/INST]
"""

#template = """Given the document and the current conversation between a user and an agent, your task is as follows: Answer any user query by using information from the document. The response should be detailed."""
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
def get_conversation_chain(vectordb):
    llm = Ollama(model="mistral")

    retriever = vectordb.as_retriever()
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(template)

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')

    conversation_chain = (ConversationalRetrievalChain.from_llm
                          (llm=llm,
                           retriever=retriever,
                           memory=memory,
                           return_source_documents=True))
    print("Conversational Chain created for the LLM using the vector store")
    return conversation_chain


### Step 5: Detect Hallucination in the LLMs Response
The `validate_answer_against_sources` function evaluates the reliability of a response by comparing it with source documents. It works as follows:

1. **Model Initialization**: Utilizes the SentenceTransformer model 'all-MiniLM-L6-v2' to generate embeddings.

2. **Threshold Setting**: Sets a similarity threshold (here, 0.5) to determine the acceptable level of similarity between the response and source documents.

3. **Extracting Source Texts**: Gathers the content of the source documents.

4. **Computing Embeddings**: Generates embeddings for both the response answer and the source texts.

5. **Calculating Similarity**: Computes cosine similarity scores between the response answer's embedding and the embeddings of each source text.

6. **Validity Check**: Checks if any of the similarity scores exceed the set threshold. If yes, it implies that the response is sufficiently similar to at least one of the source documents, suggesting its reliability, and returns `True`. If not, it returns `False`.

Essentially, this function serves as a mechanism to check the alignment of the chatbot's response with the information in the source documents, ensuring the response's accuracy and relevance.

In [7]:

def validate_answer_against_sources(response_answer, source_documents):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    similarity_threshold = 0.5  
    source_texts = [doc.page_content for doc in source_documents]

    answer_embedding = model.encode(response_answer, convert_to_tensor=True)
    source_embeddings = model.encode(source_texts, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(answer_embedding, source_embeddings)


    if any(score.item() > similarity_threshold for score in cosine_scores[0]):
        return True  

    return False  


Now that we have crafted all the necessary functions, it's time to put them into action and test their functionality.

In [8]:
content, metadata = prepare_docs(pdf_docs)


Content and metadata are extracted from the documents


In [9]:
split_docs = get_text_chunks(content, metadata)

Documents are split into 19 passages


In [10]:
vectordb=ingest_into_vectordb(split_docs)

In [11]:
conversation_chain=get_conversation_chain(vectordb)

Conversational Chain created for the LLM using the vector store


### Ask your Question

We created a conversational chain and now ready to chat with your own data. 


### Question 1

In [12]:
user_question = "who is Abonia Sojasingarayar?"
response=conversation_chain({"question": user_question})
print("Q: ",user_question)
print("A: ",response['answer'])

/Users/aboniasojasingarayar/miniconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Q:  who is Abonia Sojasingarayar?
A:   Abonia Sojasingarayar is a Machine Learning Scientist, Data Scientist, NLP Engineer, Computer Vision Engineer, AI Analyst, and Technical Writer. They have education from the Université Pondicherry in India, IA School in Boulogne-Billancourt, France, and Institut F2I in Paris, France. Abonia has certifications from IBM and deeplearning.IA, and they are proficient in various tools and techniques related to their field such as Python, TensorFlow, GCP professional data engineer Badges, Watson Assistant, and RPA (Robotic Process Automation) among others. They have worked on projects involving API integration, machine learning pipeline development, and research engineering.


We have now received an answer for a provided question. We can also view the conversation history and source documents in the response.


### Question 2

In [13]:
user_question = "where did she graduated?"
response=conversation_chain({"question": user_question})
print("Q: ",user_question)
print("A: ",response['answer'])
print("\nConversation Chain: \n",response)

Q:  where did she graduated?
A:   Abonia Sojasingarayar graduated from the Université Pondicherry in India with a licence en technologie informatique et Ingénierie degree.

Conversation Chain: 
 {'question': 'where did she graduated?', 'chat_history': [HumanMessage(content='who is Abonia Sojasingarayar?'), AIMessage(content=' Abonia Sojasingarayar is a Machine Learning Scientist, Data Scientist, NLP Engineer, Computer Vision Engineer, AI Analyst, and Technical Writer. They have education from the Université Pondicherry in India, IA School in Boulogne-Billancourt, France, and Institut F2I in Paris, France. Abonia has certifications from IBM and deeplearning.IA, and they are proficient in various tools and techniques related to their field such as Python, TensorFlow, GCP professional data engineer Badges, Watson Assistant, and RPA (Robotic Process Automation) among others. They have worked on projects involving API integration, machine learning pipeline development, and research engineer